# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846885952816                   -0.70    4.5
  2   -7.852324142757       -2.26       -1.53    1.0
  3   -7.852614395196       -3.54       -2.55    1.2
  4   -7.852645925270       -4.50       -2.88    2.5
  5   -7.852646493937       -6.25       -3.17    1.0
  6   -7.852646680045       -6.73       -4.10    1.2
  7   -7.852646686627       -8.18       -5.02    2.0
  8   -7.852646686726      -10.01       -5.57    1.5
  9   -7.852646686729      -11.52       -5.76    1.5
 10   -7.852646686730      -11.93       -6.79    1.0
 11   -7.852646686730      -13.19       -7.37    2.0
 12   -7.852646686730   +  -14.75       -7.48    2.0
 13   -7.852646686730      -15.05       -8.93    1.0
 14   -7.852646686730      -15.05       -9.32    3.0
 15   -7.852646686730   +    -Inf       -9.54    1.0
 16   -7.852646686730   +  -15.05       -9.98    1.0
 17   -7.852646686730      -14.75      -11.36 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846806971672                   -0.70           4.5
  2   -7.852527528339       -2.24       -1.64   0.80    2.2
  3   -7.852636934476       -3.96       -2.72   0.80    1.0
  4   -7.852646506920       -5.02       -3.29   0.80    2.0
  5   -7.852646675261       -6.77       -4.12   0.80    1.5
  6   -7.852646686398       -7.95       -4.83   0.80    1.8
  7   -7.852646686725       -9.49       -5.68   0.80    2.0
  8   -7.852646686730      -11.33       -6.72   0.80    1.8
  9   -7.852646686730      -12.57       -7.25   0.80    2.5
 10   -7.852646686730      -14.75       -8.05   0.80    1.0
 11   -7.852646686730   +  -14.75       -8.94   0.80    2.0
 12   -7.852646686730   +  -14.75      -10.11   0.80    2.2
 13   -7.852646686730      -15.05      -10.30   0.80    2.5
 14   -7.852646686730   +  -14.57      -11.64   0.80    1.0
 15   -7.852646686730      -14.35      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.386235e+01     3.221085e+00
 * time: 0.5090339183807373
     1     1.445648e+00     1.581162e+00
 * time: 0.7520959377288818
     2    -1.565237e+00     1.897089e+00
 * time: 0.7887539863586426
     3    -3.624514e+00     1.631327e+00
 * time: 0.8368339538574219
     4    -4.937919e+00     1.499357e+00
 * time: 0.8853549957275391
     5    -6.592419e+00     9.033809e-01
 * time: 0.9371669292449951
     6    -7.272128e+00     4.176231e-01
 * time: 0.9902658462524414
     7    -7.530774e+00     2.002258e-01
 * time: 1.0270090103149414
     8    -7.648479e+00     1.374711e-01
 * time: 1.170400857925415
     9    -7.734251e+00     1.054106e-01
 * time: 1.2049078941345215
    10    -7.782818e+00     1.050994e-01
 * time: 1.2384068965911865
    11    -7.820284e+00     6.761928e-02
 * time: 1.2724359035491943
    12    -7.837861e+00     4.966433e-02
 * time: 1.3114328384399414
    13    -7.848131e+00     2.309448e-02
 * time: 1.34794688224

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846780173014                   -0.70    4.5
  2   -7.852293238993       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686529                   -2.55
  2   -7.852646686730       -9.70       -5.67
  3   -7.852646686730   +    -Inf      -11.08
  4   -7.852646686730      -14.75      -15.43


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.3043472475548476e-13
|ρ_newton - ρ_scfv| = 3.099780334308039e-13
|ρ_newton - ρ_dm|   = 4.367863736959555e-10
